In [1]:
import folium
import geopy
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import pandas as pd
from collections import defaultdict

In [2]:
class Point:
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude

In [3]:
points = []
points.clear()
csv = pd.read_csv("Coordinates.csv")
for index, row in csv.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    points.append(Point(latitude, longitude))

In [4]:
def distance(point1, point2):
    return geodesic((point1.latitude, point1.longitude), (point2.latitude, point2.longitude)).kilometers

In [5]:
def totalDistance(order):
    totalDistance = 0
    numPoints = len(order)
    for i in range(numPoints - 1):
        point1 = order[i]
        point2 = order[i + 1]
        distance1 = distance(point1, point2)
        totalDistance += distance1
    return totalDistance

In [6]:
numPoints = len(points)
graph = [[0] * numPoints for _ in range(numPoints)]
for i in range(numPoints):
    for j in range(i + 1, numPoints):
        dist = distance(points[i], points[j])
        graph[i][j] = graph[j][i] = dist
parent = list(range(numPoints))
print(parent)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]


In [7]:
#Find MST via Kruskal's Algorithm
def kruskalMST(graph, points):
    numPoints = len(points)
    edges = []
    for i in range(numPoints):
        for j in range(i + 1, numPoints):
            if graph[i][j] > 0:
                edges.append((i, j, graph[i][j]))
    edges.sort(key = lambda edge: edge[2])
    for i in range(numPoints):
        parent[i] = i
        mst = []
    for edge in edges:
        u, v, weight = edge
        parent_u = u
        parent_v = v
        
        
        while parent[parent_u] != parent_u:
            parent_u = parent[parent_u]
        while parent[parent_v] != parent_v:
            parent_v = parent[parent_v]
        
        if parent_u != parent_v:  # Adding the edge doesn't create a cycle
            mst.append(edge)
            parent[parent_u] = parent_v
        
        if len(mst) == numPoints - 1:
            break
    return mst
mst = kruskalMST(graph, points)
print(mst)
print(len(mst))

[(13, 41, 0.09215673568837177), (10, 20, 0.1336511576731224), (6, 46, 0.1379623648353278), (7, 31, 0.16283389181285518), (3, 24, 0.20871377049243484), (0, 25, 0.23480882661865235), (35, 47, 0.2769988606777357), (16, 23, 0.3003814104546473), (15, 44, 0.30038453772375884), (17, 24, 0.3346405080525205), (2, 24, 0.34284080479027046), (4, 35, 0.4544779557380242), (8, 42, 0.45605899343109757), (39, 42, 0.4693922331003008), (29, 43, 0.515200441139487), (18, 28, 0.532281222367192), (11, 28, 0.5415895945803634), (2, 22, 0.5691754058469604), (14, 27, 0.5713776046937145), (10, 26, 0.5932944068760614), (5, 49, 0.6326523565153411), (21, 44, 0.6524064515601774), (15, 30, 0.6602542476701143), (5, 47, 0.6717650882798063), (4, 43, 0.6917718414810956), (4, 40, 0.6972268918217194), (0, 22, 0.7082919509107816), (19, 39, 0.7224162294233698), (15, 23, 0.7301684166787561), (8, 40, 0.7477410805601196), (9, 19, 0.7952759167415934), (34, 36, 0.8061103393584312), (33, 42, 0.8068600801951268), (1, 25, 0.830685912

In [8]:
Map = folium.Map(location=[points[0].latitude, points[0].longitude], zoom_start=12)

In [9]:
for edge in mst:
    u, v, weight = edge
    folium.PolyLine([(points[u].latitude, points[u].longitude), (points[v].latitude, points[v].longitude)],
                    color='blue', weight=2.5, opacity=1).add_to(Map)
count = 0
for point in points:
    folium.Marker(location = [point.latitude, point.longitude], 
                  popup = [point.latitude, point.longitude, count], 
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(Map)
    count+=1
folium.Marker(location = [points[0].latitude, points[0].longitude], 
                  popup = [points[0].latitude, points[0].longitude, 0], 
                  icon=folium.Icon(color='red', icon='map-marker')).add_to(Map)
Map

In [10]:
def oddDegreeNodes(mst):
    degrees = defaultdict(int)

    for fromNode, toNode, weight in mst:
        degrees[fromNode] += 1
        degrees[toNode] += 1

    oddNodes = set()
    for node, degree in degrees.items():
        if degree % 2 == 1:
            oddNodes.add(node)

    return oddNodes

In [11]:
oddNodes = oddDegreeNodes(mst)
print("Points with Odd Degree:", oddNodes)
print(len(oddNodes))

Points with Odd Degree: {1, 4, 7, 9, 12, 13, 14, 15, 18, 24, 29, 31, 33, 34, 36, 37, 38, 42, 43, 45, 46, 47, 49, 50}
24


In [12]:
Map = folium.Map(location=[points[0].latitude, points[0].longitude], zoom_start=12)
for point in oddNodes:
    folium.Marker(location = [points[point].latitude, points[point].longitude], 
                  popup = [points[point].latitude, points[point].longitude], 
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(Map)
Map

In [13]:
#Perfect match by linking the shortest distances between the odd nodes; suboptimal but works
#stuck here for a while due to lack of intuitive algorithm's for perfect matching
def greedyPerfectMatching(oddNodes, points):
    edges = []
    
    for i in oddNodes:
        for j in oddNodes:
            if i < j:
                dist = distance(points[i], points[j])
                edges.append((i, j, dist))
    
    # Sort edges by distance
    edges.sort(key=lambda x: x[2])
    
    matched = []
    matching = []
    
    for edge in edges:
        u, v, dist = edge
        if u not in matched and v not in matched:
            matching.append(edge)
            matched.append(u)
            matched.append(v)
    
    return matching
matched = greedyPerfectMatching(oddNodes, points)
print(matched)


Map = folium.Map(location=[points[0].latitude, points[0].longitude], zoom_start=12)

for edge in matched:
    u, v, weight = edge
    folium.PolyLine([(points[u].latitude, points[u].longitude), (points[v].latitude, points[v].longitude)],
                    color='blue', weight=2.5, opacity=1).add_to(Map)
for i in oddNodes:
    folium.Marker(location=[points[i].latitude, points[i].longitude],
                  popup=[points[i].latitude, points[i].longitude],
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(Map)
Map

[(7, 31, 0.16283389181285518), (29, 43, 0.515200441139487), (4, 47, 0.7073407423992424), (34, 36, 0.8061103393584312), (33, 42, 0.8068600801951268), (37, 50, 0.9070617595647938), (46, 49, 0.9763300840776383), (12, 13, 1.2954759296954397), (1, 24, 1.529980711932054), (14, 15, 1.896430613769071), (9, 38, 3.411728583541377), (18, 45, 11.492015505092082)]


In [14]:
#Create eulerian tour with Hierholzer's Algorithm
def createEulerianTour(multigraph, numPoints):
    graph = defaultdict(list)
    for u, v, weight in multigraph:
        graph[u].append(v)
        graph[v].append(u)
    
    #startVertex = 0
    #for u in range(numPoints):
    #    if graph[u]:
    #        startVertex = u
    #        break
    
    #if startVertex is None:
    #    return []
    
    currPath = [0]
    path = []
    
    while currPath:
        u = currPath[-1]
        if graph[u]:
            v = graph[u].pop()
            graph[v].remove(u)
            currPath.append(v)
        else:
            path.append(currPath.pop())
        
    return path[::-1]

# Create the multigraph with MST and perfect matching
multigraph = mst.copy()
for u, v, weight in matched:
    multigraph.append((u, v, weight))

eulerianTour = createEulerianTour(multigraph, numPoints)
print("Eulerian Tour:", eulerianTour)
print(len(eulerianTour))

Map = folium.Map(location=[points[0].latitude, points[0].longitude], zoom_start=12)

for i in range(len(eulerianTour) - 1):
    u = eulerianTour[i]
    v = eulerianTour[i + 1]
    folium.PolyLine([(points[u].latitude, points[u].longitude), (points[v].latitude, points[v].longitude)],
                    color='blue', weight=2.5, opacity=1).add_to(Map)
count = 0
for point in points:
    folium.Marker(location=[point.latitude, point.longitude],
                  popup=[point.latitude, point.longitude, count],
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(Map)
    count+=1

folium.Marker(location=[points[0].latitude, points[0].longitude],
              popup=[points[0].latitude, points[0].longitude],
              icon=folium.Icon(color='red', icon='map-marker')).add_to(Map)

Map

Eulerian Tour: [0, 22, 2, 24, 17, 36, 34, 36, 48, 30, 15, 14, 27, 21, 44, 15, 23, 16, 41, 13, 12, 13, 45, 18, 28, 11, 20, 10, 26, 50, 37, 50, 31, 7, 31, 32, 49, 46, 6, 49, 5, 47, 4, 40, 8, 42, 33, 42, 39, 19, 9, 38, 47, 35, 4, 43, 29, 43, 3, 24, 1, 25, 0]
63


In [15]:
# Shortcut the eulerian circuit and creating a hamiltonion circuit
def createHamiltonionCircuit(eulerianTour):
    visited = []
    hamiltonionCircuit = []
    for v in eulerianTour:
        if v not in visited:
            visited.append(v)
            hamiltonionCircuit.append(v)
    hamiltonionCircuit.append(hamiltonionCircuit[0]) #don't forget last point
    return hamiltonionCircuit

hamiltonionCircuit = createHamiltonionCircuit(eulerianTour)
print("Hamiltonion Circuit: ", hamiltonionCircuit)


Map = folium.Map(location=[points[0].latitude, points[0].longitude], zoom_start=12)

for i in range(len(hamiltonionCircuit) - 1):
    u = hamiltonionCircuit[i]
    v = hamiltonionCircuit[i + 1]
    folium.PolyLine([(points[u].latitude, points[u].longitude), (points[v].latitude, points[v].longitude)],
                    color='blue', weight=2.5, opacity=1).add_to(Map)
count = 0
for point in points:
    folium.Marker(location=[point.latitude, point.longitude],
                  popup=[point.latitude, point.longitude, count],
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(Map)
    count+=1

folium.Marker(location=[points[0].latitude, points[0].longitude],
              popup=[points[0].latitude, points[0].longitude],
              icon=folium.Icon(color='red', icon='map-marker')).add_to(Map)

Map

Hamiltonion Circuit:  [0, 22, 2, 24, 17, 36, 34, 48, 30, 15, 14, 27, 21, 44, 23, 16, 41, 13, 12, 45, 18, 28, 11, 20, 10, 26, 50, 37, 31, 7, 32, 49, 46, 6, 5, 47, 4, 40, 8, 42, 33, 39, 19, 9, 38, 35, 43, 29, 3, 1, 25, 0]
